### **TRABAJO PRÁCTICO 1 PROCESAMIENTO DEL LENGUAJE NATURAL TUIA 2024**



INTEGRANTES:

Herrera Morena (H-1187/8)

Zorzolo Rubio Juana (Z-1217/3)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/juanazorzolo/pruebacolab

Cloning into 'pruebacolab'...


In [4]:
import shutil

# Ruta de la carpeta en Google Drive
drive_folder = '/content/drive/MyDrive/NLP TP1/'

# Ruta donde está el repositorio clonado
repo_folder = '/content/pruebacolab/'

# Copiar todos los archivos de la carpeta de Drive al repositorio
shutil.copytree(drive_folder, repo_folder + 'mis_archivos')

Error: [('/content/drive/MyDrive/NLP TP1/informe nlp.gdoc', '/content/pruebacolab/mis_archivos/informe nlp.gdoc', "[Errno 95] Operation not supported: '/content/drive/MyDrive/NLP TP1/informe nlp.gdoc'"), ('/content/drive/MyDrive/NLP TP1/gutenberg_books.gsheet', '/content/pruebacolab/mis_archivos/gutenberg_books.gsheet', "[Errno 95] Operation not supported: '/content/drive/MyDrive/NLP TP1/gutenberg_books.gsheet'")]

# Importaciones

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install sentiment-analysis-spanish
!pip install transformers
!python -m spacy download en_core_web_md
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import spacy
import unicodedata
import sacremoses
import time
from ipywidgets import Text, Button, Output, VBox, HTML, Label
from ipywidgets import Text, Button, Output, VBox, HTML
from ipywidgets import Text, Button, Output, VBox, HBox
from IPython.display import display
from tabulate import tabulate
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, pipeline, MarianMTModel, MarianTokenizer
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Carga de datos

Decidimos realizar modificaciones en los nombres de columnas de los conjuntos de datos para simplificar su manejo. Este cambio facilita el manejo de variables y mejora el flujo de trabajo al parsear información.

## Juegos

In [ ]:
!gdown "1v1jO8s3g1V4wjyNp58zF3AVwJibS1kll" --output "bgg_database.csv"

Downloading...
From: https://drive.google.com/uc?id=1v1jO8s3g1V4wjyNp58zF3AVwJibS1kll
To: /content/bgg_database.csv
100% 1.83M/1.83M [00:00<00:00, 38.0MB/s]


In [ ]:
df_juegos = pd.read_csv('bgg_database.csv')
df_juegos.head()

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.415,8.60,46836.0,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,3.8776,"[{'min': 3, 'max': 4}]","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Hand Management', 'Income', 'Loans', 'Market...","['Age of Reason', 'Economic', 'Industry / Manu..."
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.377,8.53,53807.0,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,2.8308,"[{'min': 4, 'max': 4}]","['Rob Daviau', 'Matt Leacock']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Environmental', 'Medical']"
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.349,8.59,62592.0,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,3.9132,"[{'min': 3, 'max': 3}]",['Isaac Childres'],"['Action Queue', 'Action Retrieval', 'Campaign...","['Adventure', 'Exploration', 'Fantasy', 'Fight..."
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.335,8.54,44728.0,"In Ark Nova, you will plan and design a modern...",2021,1,4,90,150,14,3.7653,"[{'min': 2, 'max': 2}]",['Mathias Wigge'],"['Action Queue', 'End Game Bonuses', 'Grid Cov...","['Animals', 'Economic', 'Environmental']"
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.240,8.60,24148.0,Twilight Imperium (Fourth Edition) is a game o...,2017,3,6,240,480,14,4.3173,"[{'min': 6, 'max': 6}]","['Dane Beltrami', 'Corey Konieczka', 'Christia...","['Action Drafting', 'Area-Impulse', 'Dice Roll...","['Civilization', 'Economic', 'Exploration', 'N..."


In [ ]:
df_juegos = df_juegos.rename(columns={'game_name': 'name'})
df_juegos = df_juegos.rename(columns={'game_href': 'link'})

## Películas

In [ ]:
!gdown "1NoEtyFXuCN72QSIxR8amIcrH3X4NKy3h" --output "IMDB-Movie-Data.csv"

Downloading...
From: https://drive.google.com/uc?id=1NoEtyFXuCN72QSIxR8amIcrH3X4NKy3h
To: /content/IMDB-Movie-Data.csv
100% 309k/309k [00:00<00:00, 74.6MB/s]


In [ ]:
df_pelis = pd.read_csv('IMDB-Movie-Data.csv')
df_pelis.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40


In [ ]:
df_pelis = df_pelis.rename(columns={'Title': 'name'})
df_pelis = df_pelis.rename(columns={'Description': 'description'})
df_pelis = df_pelis.rename(columns={'Director': 'director/author'})

## Libros

In [ ]:
url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

books = []

for indice, item_libro in enumerate(soup.find_all('li')):
  # Limitar a los primeros 50 libros
  if indice >= 50:
    break

  enlace_libro = item_libro.find('a')

  # Filtra solo enlaces que contengan "/ebooks/" seguido de un número
  if enlace_libro and re.search(r'/ebooks/\d+$', enlace_libro['href']):

    # Extraer el título y el autor del texto completo en el enlace
    texto_completo = enlace_libro.text.strip()

    # Dividir el texto en título y autor
    if " de " in texto_completo:
      titulo, autor = texto_completo.split(" de ", 1)
    elif " by " in texto_completo:
      titulo, autor = texto_completo.split(" by ", 1)
    else:
      titulo = texto_completo
      autor = "Desconocido"

    # Limpiar los textos de paréntesis y espacios adicionales
    titulo = titulo.strip()
    autor = autor.split("(")[0].strip()  # Quitar cualquier número entre paréntesis

    link = 'https://www.gutenberg.org' + enlace_libro['href']

    # Obtener el género y la descripción del libro desde su página individual
    genero = "Desconocido"
    descripcion = "Desconocida"
    book_response = requests.get(link)
    book_soup = BeautifulSoup(book_response.text, 'html.parser')

    # Buscar el género en la página individual
    subject_section = book_soup.find('th', text=re.compile("Subject|Categorías", re.IGNORECASE))
    if subject_section:
      generos = subject_section.find_next('td').find_all('a')
      genero = ', '.join([g.text for g in generos])

    # Buscar la descripción del libro
    seccion_descripcion = book_soup.find('th', text=re.compile("Summary", re.IGNORECASE))
    if seccion_descripcion:
      descripcion = seccion_descripcion.find_next('td').text.strip()  # Directamente desde el <td>

    # Si no se encontró la descripción, intentar en un bloque de descripción general
    if descripcion == "Desconocida":
      descripcion_general = book_soup.find('blockquote')
      if descripcion_general:
        descripcion = descripcion_general.text.strip()

    # Agregar al diccionario de libros
    books.append({
      'title': titulo,
      'author': autor,
      'link': link,
      'genre': genero,
      'description': descripcion
    })

    # Pausa para evitar sobrecargar el servidor con solicitudes rápidas
    time.sleep(0.01)

df = pd.DataFrame(books)
df.to_csv('gutenberg_books.csv', index=False)

<ipython-input-122-60f01bddd85e>:42: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  subject_section = book_soup.find('th', text=re.compile("Subject|Categorías", re.IGNORECASE))
<ipython-input-122-60f01bddd85e>:48: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  seccion_descripcion = book_soup.find('th', text=re.compile("Summary", re.IGNORECASE))


In [ ]:
df_libros = pd.read_csv('gutenberg_books.csv')
df_libros.head()

,title,author,link,genre,description
0,呻吟語,Kun Lü,https://www.gutenberg.org/ebooks/25558,\nConduct of life\n,"""呻吟語"" by Kun Lü is a philosophical treatise wr..."
1,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,https://www.gutenberg.org/ebooks/84,\nScience fiction\n,"""Frankenstein; Or, The Modern Prometheus"" by M..."
2,"Moby Dick; Or, The Whale",Herman Melville,https://www.gutenberg.org/ebooks/2701,\nWhaling -- Fiction\n,"""Moby Dick; Or, The Whale"" by Herman Melville ..."
3,Romeo and Juliet,William Shakespeare,https://www.gutenberg.org/ebooks/1513,\nVendetta -- Drama\n,"""Romeo and Juliet"" by William Shakespeare is a..."
4,Pride and Prejudice,Jane Austen,https://www.gutenberg.org/ebooks/1342,\nEngland -- Fiction\n,"""Pride and Prejudice"" by Jane Austen is a clas..."


In [ ]:
df_libros['genre'] = df_libros['genre'].str.replace(r'\n', '', regex=True).str.strip()
df_libros = df_libros.rename(columns={'title': 'name'})
df_libros = df_libros.rename(columns={'author': 'director/author'})

# Clasificador

En base al sentimiento ingresado por el usuario, lo categoriza en un rango de 1 a 5 para definir si su estado es 'alegre', 'melancólico' o 'ni fu ni fa'.

In [ ]:
# Cargamos el tokenizador y el modelo
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizador_sentimiento = BertTokenizer.from_pretrained(model_name)
modelo_sentimiento = BertForSequenceClassification.from_pretrained(model_name)

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)

def analizar_sentimiento_usuario(frase_usuario):
  # Creamos un pipeline de clasificación
  nlp = pipeline("sentiment-analysis", model=modelo_sentimiento, tokenizer=tokenizador_sentimiento)

  # Obtener el resultado del análisis de sentimiento
  resultado_sentimiento = nlp(frase_usuario)

  # Extraer el número de estrellas de la etiqueta
  label = resultado_sentimiento[0]['label']
  score = resultado_sentimiento[0]['score']
  nro_estrellas = int(label.split()[0])

  print(f"Frase del usuario: '{frase_usuario}'")
  print(f"  Sentimiento: {label}, Score: {score:.3f}")

  return nro_estrellas

def determinar_estado_animo(nro_estrellas):
  if nro_estrellas < 3:
    return "Estado de ánimo del usuario: Melancólico"
  elif nro_estrellas == 5:
    return "Estado de ánimo del usuario: Alegre"
  else:
    return "Estado de ánimo del usuario: Ni fu ni fa"

# Parseo de texto

En base a la descripción del libro, película o juego, parsea el texto (pasa todo a minúscula, se queda con las palabras claves, saca signos de puntuación)

In [ ]:
# Cargar modelo de lenguaje y stopwords una sola vez
nlp = spacy.load("en_core_web_md")
stopwords_en = nlp.Defaults.stop_words

In [ ]:
def eliminar_acentos(texto):
  nfkd_form = unicodedata.normalize('NFKD', texto)
  return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

# Función de limpieza para manejar tanto listas como texto individual
def limpiar_texto(texto):
  if isinstance(texto, list):
    return [' '.join([
      eliminar_acentos(tok.lemma_.lower()) for tok in nlp(frase)
      if tok.lemma_ not in stopwords_en and tok.is_alpha and len(tok.lemma_) > 1
      ]) for frase in texto]
  else:
    return ' '.join([
      eliminar_acentos(tok.lemma_.lower()) for tok in nlp(texto)
      if tok.lemma_ not in stopwords_en and tok.is_alpha and len(tok.lemma_) > 1
      ])

# Función para procesar descripciones y generar embeddings en lote
def procesar_descripciones(data, columna_descripcion):
  # Limpiar texto en lote
  data['Description_clean'] = data[columna_descripcion].apply(limpiar_texto)

  # Generar embeddings en lote
  modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')
  descriptions_clean = data['Description_clean'].tolist()
  embeddings = modelo_embeddings.encode(descriptions_clean, batch_size=32)

  # Añadir los embeddings al DataFrame
  data['Description_embedding'] = [embedding.tolist() for embedding in embeddings]
  return data

"# Función para procesar descripciones y generar embeddings en lote\ndef procesar_descripciones(data, columna_descripcion):\n  # Limpiar texto en lote\n  data['Description_clean'] = data[columna_descripcion].apply(limpiar_texto)\n\n  # Generar embeddings en lote\n  modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')\n  descriptions_clean = data['Description_clean'].tolist()\n  embeddings = modelo_embeddings.encode(descriptions_clean, batch_size=32)\n\n  # Añadir los embeddings al DataFrame\n  data['Description_embedding'] = [embedding.tolist() for embedding in embeddings]\n  return data"

In [ ]:
# Aplicación a los DataFrames
df_pelis = procesar_descripciones(df_pelis, 'description')
df_juegos = procesar_descripciones(df_juegos, 'description')
df_libros = procesar_descripciones(df_libros, 'description')

# Traducción NMT

In [ ]:
# Define el modelo de traducción y el tokenizador
tokenizador_traduccion = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-es-en')
modelo_traduccion = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-es-en')

In [ ]:
def traducir_procesar_preferencia(preferencia):
  # Tokeniza el texto y genera la traducción
  inputs = tokenizador_traduccion(preferencia, return_tensors="pt")
  outputs = modelo_traduccion.generate(**inputs)
  preferencia_ingles = tokenizador_traduccion.decode(outputs[0], skip_special_tokens=True)

  # Parsea del texto y genera los embeddings
  preferencia_ingles_limpia = limpiar_texto(preferencia_ingles)

  modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')
  preferencia_embedding = modelo_embeddings.encode(preferencia_ingles_limpia)

  return preferencia_ingles, preferencia_ingles_limpia, preferencia_embedding

# Recomendación de opciones

In [ ]:
def similitud_coseno(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    distancia = 1 - (dot_product / (norm_A * norm_B)) if norm_A and norm_B else float('inf')
    return distancia

def obtener_top_3(data, preferencia_embedding):
    lista_distancias = []

    # Iterar sobre el data para calcular la distancia
    for indice, fila in data.iterrows():
        distancia = similitud_coseno(preferencia_embedding, np.array(fila['Description_embedding']))
        lista_distancias.append({'index': indice, 'name': fila['name'], 'distance': distancia})

    # Crear un DataFrame con las distancias ordenadas
    data_con_distancias = pd.DataFrame(lista_distancias)
    data_con_distancias = data_con_distancias.sort_values(by='distance').head(3)

    # Filtrar las columnas disponibles y devolver los nombres y otros detalles
    columnas_deseadas = ['name', 'director/author', 'link']
    columnas_disponibles = [col for col in columnas_deseadas if col in data.columns]
    data_sugerencias = data.loc[data_con_distancias['index'], columnas_disponibles].reset_index(drop=True)

    return data_sugerencias

def mostrar_top_tres(top_data):
    print(tabulate(top_data, headers='keys', tablefmt='pretty', showindex=False))

# Programa principal

Con base en el estado de ánimo del usuario y la frase ingresada, el programa ofrece recomendaciones pertinentes entre películas, juegos de mesa o libros.

# sin ner

In [ ]:
def programa_principal(b):
  with output:
    output.clear_output()
    frase_usuario = frase_input.value
    nro_estrellas = analizar_sentimiento_usuario(frase_usuario)
    determinar_estado_animo(nro_estrellas)

    print("-"*50)

    preferencia = preferencia_input.value
    preferencia_ingles, preferencia_ingles_limpia, preferencia_embedding = traducir_procesar_preferencia(preferencia)

    print(f"Preferencia ingresada: '{preferencia}'")
    print("  Traducción:", preferencia_ingles)
    print("  Traducción limpia:", preferencia_ingles_limpia)

    top_pelis = obtener_top_3(df_pelis, preferencia_embedding)
    top_juegos = obtener_top_3(df_juegos, preferencia_embedding)
    top_libros = obtener_top_3(df_libros, preferencia_embedding)

    # Mostrar resultados
    print("\nTop 3 películas sugeridas:")
    mostrar_top_tres(top_pelis)

    print("\nTop 3 juegos sugeridos:")
    mostrar_top_tres(top_juegos)

    print("\nTop 3 libros sugeridos:")
    mostrar_top_tres(top_libros)

def verificar_y_ejecutar(b):
  mensaje_error.value = ""
  # Verificar si ambos campos están completos
  if not frase_input.value.strip() or not preferencia_input.value.strip():
    mensaje_error.value = "<span style='color: red;'>Por favor, completar todos los campos.</span>"
  else:
    programa_principal(boton_recomendar)

In [ ]:
# Definir widgets de entrada
frase_label = Label(value="¿Cómo te sentís hoy? :")
frase_input = Text(placeholder='Ejemplo: increíble', layout={'width': '60%', 'height': '10'})

preferencia_label = Label(value="¿Tenés alguna temática preferida? :")
preferencia_input = Text(placeholder='Ejemplo: comedia romántica', layout={'width': '60%', 'height': '10'})

# Botón y salida
boton_recomendar = Button(description="Obtener Recomendaciones", layout={'width': '15%'})
output = Output()
mensaje_error = HTML(value="")

# Conectar el evento de teclado al botón
frase_input.on_submit(verificar_y_ejecutar)
preferencia_input.on_submit(verificar_y_ejecutar)

# Mostrar los widgets
display(VBox([frase_label, frase_input, preferencia_label, preferencia_input, boton_recomendar, mensaje_error, output]))